### Lipidtools

Utility classes for creating a new topology (.itp) file, making a sample minimal membrane containing that species, minimizing that system, generating and showing an image of that species

Add the following to the top of any Notebook to be able to use these tools  
>**%run Lipidtools.ipynb**

 Class **lipid**(name, head, link, alpha, beta,descr).write(fileName) : **lipid** 
   method **lipid**.create(verbose) : **lipid**  
   method **lipid**.appendTo(fileName) : **lipid**  
   method **lipid**.buildMembrane(name) : **lipid**  
   method **lipid**.minimize(verbose,parameterFile) : **lipid** 
   method **lipid**.show(x,y,z) : **lipid**  

* name: Martini notation for the lipid, eg: "POPC"
* head: Martini notation for the head group, eg: "C P", "P E", or "PI"
* link: Martini notation for the linking groups, eg: "G G" or "A A" 
* alpha: Martini notation for the alpha acyl tail, eg: "CDCC" or oleic  
* beta: Martini notation for the beta acyl tail, eg: "CCCC" for palmitic
* description: set the description for the species
* methods 
    * **create** creates a topology, depending on "verbose" outputs to the stream a summary or the output of that process, and returns an instance to this lipid
    * **write**(fileName) writes the topology to fileName.itp, returns an instance to this lipid
    * **append**(fileName) appends the topology to fileName.itp (removing any previous instance of this name), returns an instance to this lipid
    * **buildMembrane**(Name) creates a minimal membrane system name.gro containing this species
    * **minimize**(verbose,parameterFile) if a membrane system doesn't exist it makes one, and then minimzes it using the parameter (.mdp) file and writes a summary (if verbose=false) or the complete output in the stream, returns an instance to this lipid
    * method show(x,y,z) writes to output stream an image of the species using x,y,z to translate (move) the image in the camera field  
* fields 
    * topology : string
    * success : boolean - whether minimize completed all steps

**USAGE:** lipid("POPC","C P","G G","CDCC","CCCC").description("A general model phosphatidylcholine (PC) lipid corresponding to atomistic e.g. C16:0/18:1 1-palmitoyl-2-oleoyl (POPC) tails.").appendTo("./martini.ff/myLipids.itp").minimize(false,"Test.mdp").show().success() 

Will add the lipid to myLipids.itp, print out the summary result of minimizing a membrane containing this lipid using Test.mdp, and an image of the lipid, and return if the creation/embeding/minimization was successfull.

In [68]:
# import libraries used in these utilities
import os        # Operating system specific commands
import re        # Regular expression library
import datetime  # Date and Time routines
import weakref   # Object lifetime managment
import tempfile  # Temporary paths and files
import shutil    # Shell utilities
import fileinput # input output library

class StopExecution(Exception):  #Exception class for an imediate stop
    def _render_traceback_(self):
        pass

#defaults
defaultpath="./"
defaultmdp = os.path.join(defaultpath,"test.mdp")
defaultmartini = os.path.join(defaultpath,"martini.ff")
defaultinsane = os.path.join(defaultpath,"insane+SF.py")

#verify defaults exist
if not os.path.isfile(defaultmdp): 
    print("WARNING: default MDP [{}] missing".format(defaultmdp))
if not os.path.isfile(defaultinsane): 
    print("WARNING: default insane [{}] missing".format(defaultinsane))
if not os.path.isdir(defaultmartini): 
    print("WARNING: default martini path [{}] missing".format(defaultmartini))

topologyGenerator = defaultmartini+'/lipid-martini-itp-v06.py'    
if not os.path.isfile(topologyGenerator):
    print('Missing martini lipid topology generator:'+topologyGenerator)
    raise StopExecution

In [79]:
class lipid:
    def __init__(self,name,head,link,alpha,beta,descr):
        if not name:
            print("usage lipids(name,head,link,alpha,beta)")
        self.temp = tempfile.mkdtemp()
        self._finalizer = weakref.finalize(self, shutil.rmtree, self.temp)
        self.name=name
        self.head=head
        self.link=link
        self.alpha = alpha
        self.beta = beta
        self.topology = ""
        self.success = False
        self.parameterization = ";   This topology follows the standard Martini 2.0 lipid definitions and building block rules.\n; Reference(s): \n;   S.J. Marrink, A.H. de Vries, A.E. Mark. Coarse grained model for semi-quantitative lipid simulations. JPC-B, 108:750-760, \n;   2004. doi:10.1021/jp036508g \n;   S.J. Marrink, H.J. Risselada, S. Yefimov, D.P. Tieleman, A.H. de Vries. The MARTINI force field: coarse grained model for \n;   biomolecular simulations. JPC-B, 111:7812-7824, 2007. doi:10.1021/jp071097f \n;   T.A. Wassenaar, H.I. Ingolfsson, R.A. Bockmann, D.P. Tieleman, S.J. Marrink. Computational lipidomics with insane: a versatile \n;   tool for generating custom membranes for molecular simulations. JCTC, 150410125128004, 2015. doi:10.1021/acs.jctc.5b00209\n; Created: "+datetime.datetime.now().strftime("%Y.%m.%d")
        self.description = descr    
        self.membrane=""
    def remove(self):
        self._finalizer()
    @property
    def removed(self):
        return not self._finalizer.alive        
    def create(self,verbose):
        if not os.path.isfile(topologyGenerator):
            print('Missing martini lipid topology generator:')
            raise StopExecution
        generation = !python {topologyGenerator} -o {self.temp}/{self.name}.itp -alname {self.name} -name {self.name} -alhead '{self.head}' -allink '{self.link}' -altail '{self.alpha + " " + self.beta}'
        if verbose:
            print(generation)
        else:
            print("created "+self.temp+"/"+self.name+".itp")
       
        #update description and parameters
        with fileinput.FileInput(self.temp+"/"+self.name+".itp", inplace=True) as file:
            for line in file:
                if line == ";   This is a ...\n":
                    print(self.description, end='')
                elif line == ";   Was modeled on ...\n":
                    print(self.parameterization, end='')
                else:
                    print(line, end='')
        return self
    def appendTo(self,filename):
        if not os.path.isfile(self.temp+"/"+self.name+".itp"):
            self.create(False)
        if not os.path.isfile(self.temp+"/"+self.name+".itp"):
            print('Missing martini lipid topology generator:')
            raise StopExecution
        # remove topology specifying this lipid
        # add the topology in the temp directory
        return self
    def buildMembrane(self,name):
        if not os.path.isfile(self.temp+"/"+self.name+".itp"):
            self.create(False)
        if not os.path.isfile(self.temp+"/"+self.name+".itp"):
            print('Missing martini lipid topology generator:')
            raise StopExecution
        self.membrane = name
        # build a simple membrane to visualize this species
        insane = defaultinsane
        !python2 {insane} -o {self.membrane}.gro -p {self.membrane}.top -d 0 -x 3 -y 3 -z 3 -sol PW -center -charge 0 -orient -u {self.name}:1 -l {self.name}:1 -itpPath {martinipath}
        if os.path.isfile(self.temp+"/"+self.membrane+".gro"):
            print("Membrane "+self.membrane+" created")
        else:
            print("Membrane "+self.membrane+" not created")
        return self
    def minimize(self,parameterFile,verbose):
        if not self.membrane:
            self.buildMembrane('default')
            self.membrane = 'default'
        return self
    def show(self,x=0,y=0,z=0):
        return self
    def success(self):
        return self.created
    def __repr__(self):
        return "Lipid ({})".format(self.name)
        


# Tests 

NB: these will only run when the notebook is loaded directly and run, they will not run when the notebook is loaded as a %run module from other notebooks  
Run this notebook stand alone to run all tests 

In [80]:
Lipid = lipid("POPC","C P","G G","CDCC","CCCC","A general model phosphatidylcholine (PC) lipid corresponding to atomistic e.g. C16:0/18:1 1-palmitoyl-2-oleoyl (POPC) tails.")
#Lipid.create(True)
#Lipid.appendTo("./martini.ff/myLipids.itp")
Lipid.minimize("Test.mdp",False)
#Lipid.show()

created /tmp/tmptj20x9rl/POPC.itp
; X: 3.000 (4 lipids) Y: 3.000 (4 lipids)
; 16 lipids in upper leaflet, 16 lipids in lower leaflet
Traceback (most recent call last):
  File "insane+SF.py", line 1324, in <module>
    atoms    = zip(lipidsa[lipid][1].split(),lipidsx[lipidsa[lipid][0]],lipidsy[lipidsa[lipid][0]],lipidsz[lipidsa[lipid][0]])
KeyError: '{self.name}'
Membrane default not created


Lipid (POPC)

In [2]:
import datetime
lipidname = "OIPC"
tail = "CDDC CDCC" 
link = "G G"
head = "C P"

description = ";   A general model phosphatidylcholine (PC) lipid \n;      C18:1(9c) oleic acid, and C18:2(9c,12c) linoleic acid\n"
modeledOn=";   This topology follows the standard Martini 2.0 lipid definitions and building block rules.\n; Reference(s): \n;   S.J. Marrink, A.H. de Vries, A.E. Mark. Coarse grained model for semi-quantitative lipid simulations. JPC-B, 108:750-760, \n;   2004. doi:10.1021/jp036508g \n;   S.J. Marrink, H.J. Risselada, S. Yefimov, D.P. Tieleman, A.H. de Vries. The MARTINI force field: coarse grained model for \n;   biomolecular simulations. JPC-B, 111:7812-7824, 2007. doi:10.1021/jp071097f \n;   T.A. Wassenaar, H.I. Ingolfsson, R.A. Bockmann, D.P. Tieleman, S.J. Marrink. Computational lipidomics with insane: a versatile \n;   tool for generating custom membranes for molecular simulations. JCTC, 150410125128004, 2015. doi:10.1021/acs.jctc.5b00209\n; Created: "

now = datetime.datetime.now()
membrane="testmembrane"
insane="../insane+SF.py"
mdparams="../test.mdp"
martinipath="../martini.ff/"
ITPCatalogue="./epithelial.cat"
ITPMasterFile="martini_v2_epithelial.itp"

modeledOn+= now.strftime("%Y.%m.%d")+"\n"

# Cleaning up intermediate files from previous runs
!rm -f *#*
!rm -f *step*
!rm -f {membrane}*

In [33]:
import fileinput
import os.path

print("Create itp")
!python {martinipath}/lipid-martini-itp-v06.py -o {lipidname}.itp -alname {lipidname} -name {lipidname} -alhead '{head}' -allink '{link}' -altail '{tail}'

#update description and parameters
with fileinput.FileInput(lipidname+".itp", inplace=True) as file:
    for line in file:
        if line == ";   This is a ...\n":
            print(description, end='')
        elif line == ";   Was modeled on ...\n":
            print(modeledOn, end='')
        else:
            print(line, end='')



Create itp
The Martini lipid itp generator version 0.6  Args are: -o OIPC.itp -alname OIPC -alhead 'C P' -allink 'G G' -altail 'CDDC CDCC'


In [34]:
#Add this ITP file to the catalogue file
if not os.path.exists(ITPCatalogue):
    ITPCatalogueData = []
else:
    with open(ITPCatalogue, 'r') as file :
        ITPCatalogueData = file.read().splitlines()

    ITPCatalogueData = [x for x in ITPCatalogueData if not x==lipidname+".itp"]

ITPCatalogueData.append(lipidname+".itp")
    
with open(ITPCatalogue, 'w') as file :
    file.writelines("%s\n" % item for item in ITPCatalogueData)
    
#build ITPFile    
with open(martinipath+ITPMasterFile, 'w') as masterfile:
    for ITPfilename in ITPCatalogueData:
        with open(ITPfilename, 'r') as ITPfile :
            for line in ITPfile:
                masterfile.write(line)
    
print("Done")

Done


In [35]:
# build a simple membrane to visualize this species
!python2 {insane} -o {membrane}.gro -p {membrane}.top -d 0 -x 3 -y 3 -z 3 -sol PW -center -charge 0 -orient -u {lipidname}:1 -l {lipidname}:1 -itpPath {martinipath}

; X: 3.000 (4 lipids) Y: 3.000 (4 lipids)
; 16 lipids in upper leaflet, 16 lipids in lower leaflet
; NDX Solute 1 0
; Charge of protein: 0.000000
; NDX Membrane 1 384
; Charge of membrane: 0.000000
; Total charge: 0.000000
; NDX Solvent 385 0
; NDX System 1 384


In [36]:
import os #Operating system specific commands
import re #Regular expression library

print("Test")
print("Grompp")
grompp = !gmx grompp -f {mdparams} -c {membrane}.gro -p {membrane}.top -o  {membrane}.tpr
success=True
for line in grompp:
    if re.search("ERROR", line):
        success=False
    if re.search("Fatal error", line):
        success=False
    #if not success:
    print(line)
    
if success:
    print("Run")
    !export GMX_MAXCONSTRWARN=-1
    !export GMX_SUPPRESS_DUMP=1
    run = !gmx mdrun -v -deffnm {membrane}
    summary=""
    logfile = membrane+".log"
    if not os.path.exists(logfile):
        print("no log file")
        print("== === ====")
        for line in run:
            print(line)
    else:
        try:
            file = open(logfile, "r")
            fe = False
            for line in file:
                if fe:
                    success=False
                    summary=line
                elif re.search("^Steepest Descents.*converge", line):
                    success=True
                    summary=line
                    break
                elif re.search("Fatal error", line):
                    fe = True
        except IOError as exc:
            sucess=False;
            summary=exc;
    if success:
        print("Success")
    else:
        print(summary)

Test
Grompp
                      :-) GROMACS - gmx grompp, 2018.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar    Aldert van Buuren   Rudi van Drunen     Anton Feenstra  
  Gerrit Groenhof    Aleksei Iupinov   Christoph Junghans   Anca Hamuraru   
 Vincent Hindriksen Dimitrios Karkoulis    Peter Kasson        Jiri Kraus    
  Carsten Kutzner      Per Larsson      Justin A. Lemkul    Viveca Lindahl  
  Magnus Lundborg   Pieter Meulenhoff    Erik Marklund      Teemu Murtola   
    Szilard Pall       Sander Pronk      Roland Schulz     Alexey Shvetsov  
   Michael Shirts     Alfons Sijbers     Peter Tieleman    Teemu Virolainen 
 Christian Wennberg    Maarten Wolf   
                           and the project leaders:
        Mark Abraham, Berk Hess, Erik Lindahl, and David van der Spoel

Copyright (c) 1991-2000, University of Groningen, The Netherlands.
Copyright (c) 2001-2017, Th